In [1]:
import re
import requests
import mercury as mr
import ollama
import html as _html
from IPython.display import HTML

In [2]:
import os
from ollama import Client
from dotenv import load_dotenv
load_dotenv(".ollama.env")
client = Client(
    host="https://ollama.com",
    headers={'Authorization': 'Bearer ' + os.environ.get('OLLAMA_API_KEY')}
)

In [3]:
SYSTEM_PROMPT = """
You are an AI Web Designer.

Return ONLY a complete, standalone HTML document.
Do not use markdown. Do not use JSON. Do not add explanations.

IMAGE RULES (VERY IMPORTANT):
- Use ONLY publicly accessible image URLs.
- Prefer these sources (in order):
  1. https://picsum.photos
  2. https://images.unsplash.com
- Do NOT invent image domains.
- Do NOT use placeholder domains like example.com for images.
- Every <img> tag MUST have a valid absolute https:// URL.
- Images must load without authentication or API keys.

If using picsum.photos:
- Use URLs like: https://picsum.photos/seed/any-text/800/500

If using Unsplash:
- Use direct image URLs starting with:
  https://images.unsplash.com/
- Include width parameters (e.g. ?w=1200)

HTML RULES:
- Output must start with <!doctype html>.
- Include <html>, <head>, and <body>.
- Include <meta charset="utf-8">.
- Inline CSS is allowed.
- Inline JavaScript is allowed.
- Use semantic HTML where possible.

CONTENT RULES:
- The page should look visually complete.
- Use real images, not broken links.
- Do not include comments or placeholder text like "image here".

Return ONLY the HTML document.
"""

In [4]:
def extract_html(text: str) -> str:
    if not text:
        return "<!doctype html><html><body></body></html>"
    t = text.strip()

    # jeśli jednak pojawią się fence (awaryjnie)
    m = re.search(r"```(?:html)?\s*(.*?)\s*```", t, re.DOTALL | re.IGNORECASE)
    if m:
        t = m.group(1).strip()

    # znajdź początek HTML
    idx = t.lower().find("<!doctype html")
    if idx == -1:
        idx = t.lower().find("<html")
    if idx != -1:
        t = t[idx:]

    # przytnij do końca </html> jeśli jest
    end = t.lower().rfind("</html>")
    if end != -1:
        t = t[: end + len("</html>")]

    return t


In [5]:
def iframe_srcdoc(html_doc: str, height="1024px") -> str:
    srcdoc = _html.escape(html_doc, quote=True)
    return f"""
<iframe
  srcdoc="{srcdoc}"
  style="width:100%; height:{height}; border:1px solid #ddd; border-radius:8px;"
  sandbox="allow-scripts allow-forms allow-same-origin"
></iframe>
""".strip()

In [6]:
left, right = mr.Columns(2)

ColumnsBox(children=(ColumnOutput(layout=Layout(flex='1 1 0px', min_width='100px'), _dom_classes=('mljar-colum…

In [7]:
with right:
    preview, code = mr.Tabs(["Preview", "Code"])

In [8]:
with left:
    chat = mr.Chat()

In [9]:

messages = [{"role": "system", "content": SYSTEM_PROMPT}]


In [10]:
prompt = mr.ChatInput(placeholder="Describe the website you want ...")


In [13]:

if prompt.value:
    # user message
    chat.add(mr.Message(markdown=prompt.value, role="user"))
    messages.append({"role": "user", "content": prompt.value})

    # call local LLM (Ollama)
    stream = client.chat( # ollama.chat
        model="gpt-oss:120b",
        messages=messages,
        stream=True,
        think='low'
    )

    ai_msg = mr.Message(role="assistant", emoji="🤖")
    chat.add(ai_msg)

    thinking, content = "", ""
    tool_calls = []

    code.clear()
    for chunk in stream:
        if getattr(chunk.message, "thinking", None):
            thinking += chunk.message.thinking
            ai_msg.append_markdown(chunk.message.thinking)

        if getattr(chunk.message, "content", None):
            content += chunk.message.content

            with code:
                print(chunk.message.content, flush="", end="")
            
    messages.append({
        "role": "assistant",
        "thinking": thinking,
        "content": content
    })

    last_html = extract_html(content)
    with preview:
        preview.clear()
        display(HTML(iframe_srcdoc(last_html)))
    with code:
        code.clear()
        print(last_html)